Universidad del Valle de Guatemala <br>
Security Data Science <br>
Jorge Yass <br>
Douglas de León - 18037 <br>
Augusto Alonso - 181085 <br>

# Laboratorio 6 - Hunting with Data Science

## Parte 1 - Filtrado y preprocesamiento

In [26]:
import json
import pickle
from flare.data_science.features import domain_tld_extract
import pandas as pd
import numpy as np
from flare.tools.alexa import Alexa
from flare.tools.umbrella import Umbrella
from flare.tools.whoisip import WhoisLookup
from flare.data_science.features import entropy
import whois

In [2]:
data = []
with open("large_eve.json") as file:
    for line in file.readlines():
        data.append(json.loads(line))

In [3]:
print("Cantidad de registros:", "{:,}".format(len(data)))

Cantidad de registros: 746,909


In [4]:
only_dns = list(filter(lambda x: "dns" in x, data))

In [5]:
print("Cantidad de registros DNS: ", "{:,}".format(len(only_dns)))

Cantidad de registros DNS:  15,749


In [6]:
print("Ejemplos: \n")
for ej in only_dns[:2]:
    print(json.dumps(ej, indent=4))

Ejemplos: 

{
    "timestamp": "2017-07-22T17:33:16.661646-0500",
    "flow_id": 1327836194150542,
    "pcap_cnt": 22269,
    "event_type": "dns",
    "vlan": 110,
    "src_ip": "2001:0dbb:0c18:0011:0260:6eff:fe30:0863",
    "src_port": 59680,
    "dest_ip": "2001:0500:0001:0000:0000:0000:803f:0235",
    "dest_port": 53,
    "proto": "UDP",
    "dns": {
        "type": "query",
        "id": 15529,
        "rrname": "api.wunderground.com",
        "rrtype": "A",
        "tx_id": 0
    }
}
{
    "timestamp": "2017-07-22T17:33:24.990320-0500",
    "flow_id": 2022925111925872,
    "pcap_cnt": 54352,
    "event_type": "dns",
    "vlan": 110,
    "src_ip": "2001:0dbb:0c18:0011:0260:6eff:fe30:0863",
    "src_port": 38051,
    "dest_ip": "2001:0500:0003:0000:0000:0000:0000:0042",
    "dest_port": 53,
    "proto": "UDP",
    "dns": {
        "type": "query",
        "id": 58278,
        "rrname": "stork79.dropbox.com",
        "rrtype": "A",
        "tx_id": 0
    }
}


In [7]:
df = pd.json_normalize(data=only_dns)
df.head()

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
3,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,NaN,NaN,NXDOMAIN,NaN,NaN
4,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,<root>,SOA,NaN,NXDOMAIN,20864.0,NaN


In [8]:
A_type = df[df['dns.rrtype'] == 'A']
print(len(A_type))
A_type.head()

2849


,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
5,2017-07-22T17:33:36.672785-0500,237919524635665,55496,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,41663,2001:07fd:0000:0000:0000:0000:0000:0001,53,UDP,query,45082,api.wunderground.com,A,0.0,NaN,NaN,NaN
6,2017-07-22T17:33:38.537426-0500,2167545251640146,55687,dns,180,192.168.198.62,35092,192.168.207.4,53,UDP,query,7425,safebrowsing.clients.google.com.home,A,0.0,NaN,NaN,NaN


In [9]:
A_type['dns.rrname'].unique()

array(['api.wunderground.com', 'stork79.dropbox.com',
       'hpca-tier2.office.aol.com.ad.aol.aoltw.net',
       'safebrowsing.clients.google.com.home', 'fxfeeds.mozilla.com',
       'www.metasploit.com', 'aolmtcmxm03.office.aol.com',
       'aolmtcmxm02.office.aol.com.ad.aol.aoltw.net',
       'aolmtcmxm02.office.aol.com', 'hpca-tier2.office.aol.com',
       'aolmtcmxm03.office.aol.com.ad.aol.aoltw.net',
       'aolmtcmxm04.office.aol.com', 'safebrowsing.clients.google.com',
       'wpad.home', 'safebrowsing.clients.google.com.stayonline.net',
       'aolmtcmxm04.office.aol.com.ad.aol.aoltw.net',
       'AOLDTCMA04.ad.aol.aoltw.net.office.aol.com',
       'AOLDTCMA04.office.aol.com', '192.168.22.110phpmyadmin',
       'secure.informaction.com', 'secure.informaction.com.localdomain',
       'safebrowsing.clients.google.com.localdomain', 'ueip.vmware.com',
       'en-us.fxfeeds.mozilla.com',
       '192.168.22.110phpmyadmin.localdomain', 'time.windows.com',
       'softwareupdate.vmwar

In [10]:
df = pd.DataFrame(np.array([domain_tld_extract(x) for x in A_type['dns.rrname'].unique()]), columns=['domain'])
df.head()

clean_df = df.copy(deep=True)

## Parte 2 - Data Science

In [11]:
from caracteristicasDerivadas import H_entropy, proporcionVocalesConsonantes, posicionPrimerDigito

In [12]:
df['longitud'] = df['domain'].str.len()
df['digitos'] = df['domain'].str.count('[0-9]')
df['entropia'] = df['domain'].apply(H_entropy)
df['proporcionVocalesConsonantes'] = df['domain'].apply(proporcionVocalesConsonantes)
df['posicionPrimerDigito'] = df['domain'].apply(posicionPrimerDigito)

df.drop(['domain'], axis=1, inplace=True)

In [13]:
# Cargar clasificador DGA de laboratorio 1
with open('classifier.pickle', 'rb') as handle:
    clf = pickle.load(handle)

In [14]:
preds = clf.predict(df)

In [15]:
clean_df['pred'] = preds
clean_df

,domain,pred
0,wunderground.com,0
1,dropbox.com,0
2,aoltw.net,0
3,safebrowsing.clients.google.com.home,1
4,mozilla.com,0
...,...,...
172,ntp.ubuntu.com.localdomain,0
173,microsoft.com,1
174,verisign.com,0
175,192.168.22.201:.stayonline.net,1


## Parte 3 - Dominio de experto

In [16]:
#https://umbrella.cisco.com/blog/cisco-umbrella-1-million
umbrellaCisco = Umbrella(limit=1000000)

[+] Updated Umbrella Top 1 Million list...


In [17]:
only_dga = clean_df[clean_df['pred'] == 1]

In [18]:
only_dga

,domain,pred
3,safebrowsing.clients.google.com.home,1
18,192.168.22.110phpmyadmin,1
21,safebrowsing.clients.google.com.localdomain,1
24,192.168.22.110phpmyadmin.localdomain,1
27,ntkrnlpa.info,1
28,portswigger.net,1
34,sql-ledger.org,1
35,backtrack-linux.org,1
43,safebrowsing.clients.google.com.hackerlabs.vpn,1
48,phpmyadmin.net,1


In [19]:
# Los que se encuentran en el top 1,000,000
in_top_million = only_dga[only_dga.apply(lambda x: umbrellaCisco.domain_in_umbrella(x['domain']), axis=1)]
# Los que no
not_in_top_million = only_dga[only_dga.apply(lambda x: not umbrellaCisco.domain_in_umbrella(x['domain']), axis=1)]

In [20]:
in_top_million.head()

,domain,pred
28,portswigger.net,1
48,phpmyadmin.net,1
51,microsoft.com,1
60,postgresql.org,1
75,microsoft.com,1


In [21]:
not_in_top_million

,domain,pred
3,safebrowsing.clients.google.com.home,1
18,192.168.22.110phpmyadmin,1
21,safebrowsing.clients.google.com.localdomain,1
24,192.168.22.110phpmyadmin.localdomain,1
27,ntkrnlpa.info,1
34,sql-ledger.org,1
35,backtrack-linux.org,1
43,safebrowsing.clients.google.com.hackerlabs.vpn,1
63,secure.informaction.com.hackerlabs.vpn,1
67,safebrowsing.clients.google.com.lan,1


#### Retornar fecha de creación para los que no estan en el top 1M

In [35]:
def add_created_at(x):
    try:
        w = whois.query(x)
        return w.creation_date
    except:
        return None

In [36]:
not_in_top_million['created_at'] = not_in_top_million['domain'].apply(lambda x: add_created_at(x))

Unknown TLD: .home
Valid TLDs: .ac_uk .am .amsterdam .ar .at .au .bank .be .biz .br .by .ca .cc .cl .club .cn .co .co_il .co_jp .com .com_au .com_tr .cr .cz .de .download .edu .education .eu .fi .fm .fr .frl .game .global_ .hk .id_ .ie .im .in_ .info .ink .io .ir .is_ .it .jp .kr .kz .link .lt .lv .me .mobi .mu .mx .name .net .ninja .nl .nu .nyc .nz .online .org .pe .pharmacy .pl .press .pro .pt .pub .pw .rest .ru .ru_rf .rw .sale .se .security .sh .site .space .store .tech .tel .theatre .tickets .trade .tv .ua .uk .us .uz .video .website .wiki .work .xyz .za Unknown TLD: .110phpmyadmin
Valid TLDs: .ac_uk .am .amsterdam .ar .at .au .bank .be .biz .br .by .ca .cc .cl .club .cn .co .co_il .co_jp .com .com_au .com_tr .cr .cz .de .download .edu .education .eu .fi .fm .fr .frl .game .global_ .hk .id_ .ie .im .in_ .info .ink .io .ir .is_ .it .jp .kr .kz .link .lt .lv .me .mobi .mu .mx .name .net .ninja .nl .nu .nyc .nz .online .org .pe .pharmacy .pl .press .pro .pt .pub .pw .rest .ru .ru_rf 

/usr/local/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [37]:
not_in_top_million['created_at']

3                     NaT
18                    NaT
21                    NaT
24                    NaT
27    2021-09-20 18:01:53
34    2000-09-08 16:25:09
35    2009-04-29 00:26:42
43                    NaT
63                    NaT
67                    NaT
71                    NaT
88                    NaT
135                   NaT
160                   NaT
163                   NaT
165                   NaT
166                   NaT
167                   NaT
168                   NaT
169                   NaT
175   1999-12-14 19:57:16
Name: created_at, dtype: datetime64[ns]

In [42]:
DGA_list = not_in_top_million[not_in_top_million.created_at.isnull()]
DGA_list

,domain,pred,created_at
3,safebrowsing.clients.google.com.home,1,NaT
18,192.168.22.110phpmyadmin,1,NaT
21,safebrowsing.clients.google.com.localdomain,1,NaT
24,192.168.22.110phpmyadmin.localdomain,1,NaT
43,safebrowsing.clients.google.com.hackerlabs.vpn,1,NaT
63,secure.informaction.com.hackerlabs.vpn,1,NaT
67,safebrowsing.clients.google.com.lan,1,NaT
71,"""192.168.206.56""",1,NaT
88,192.168.26-27.0,1,NaT
135,192.168.21.1201,1,NaT


In [50]:
print("Los dominios que consideramos realmente sopechosos son los siguientes ya que no estaban\nen el top 1M ni se encontro su fecha de creación:\n\n")
for domain in DGA_list['domain']:
    print(domain)

Los dominios que consideramos realmente sopechosos son los siguientes ya que no estaban
en el top 1M ni se encontro su fecha de creación:


safebrowsing.clients.google.com.home
192.168.22.110phpmyadmin
safebrowsing.clients.google.com.localdomain
192.168.22.110phpmyadmin.localdomain
safebrowsing.clients.google.com.hackerlabs.vpn
secure.informaction.com.hackerlabs.vpn
safebrowsing.clients.google.com.lan
"192.168.206.56"
192.168.26-27.0
192.168.21.1201
vtlfccmfxlkgifuf.com
1922.168.22.254
1922.168.22.254.home
192.168.21-28.0
ejfodfmfxlkgifuf.xyz
192.168.21-28.0.home
192.168.22.201:


In [51]:
print("Los dominios que consideramos sopechosos son los siguientes ya que no estaban\nen el top 1M pero si se encontro su fecha de creación:\n\n")
for index, row in not_in_top_million[not_in_top_million.created_at.notnull()].iterrows():
        print(row.domain, " creado el ", row.created_at)

Los dominios que consideramos sopechosos son los siguientes ya que no estaban
en el top 1M pero si se encontro su fecha de creación:


ntkrnlpa.info  creado el  2021-09-20 18:01:53
sql-ledger.org  creado el  2000-09-08 16:25:09
backtrack-linux.org  creado el  2009-04-29 00:26:42
192.168.22.201:.stayonline.net  creado el  1999-12-14 19:57:16
